# Imports

In [2]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

# Neural Network
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import pickle

In [3]:
# Import Data
df = pd.read_csv('data/train-test/app_train.csv')

# Quick Details

In [4]:
df.shape

(3925778, 55)

In [6]:
df.describe()

,pitcher_id,batter_id,release_speed,release_pos_x,release_pos_z,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,inning,inning_topbot,effective_speed,release_spin_rate,release_extension,Catcher,FirstBasemen,SecondBasemen,ThirdBasemen,ShortStop,LeftField,CenterField,RightField,at_bat_number,pitch_number,bat_score,fld_score,post_bat_score,post_fld_score,vx0,vy0,vz0,ax,ay,az,outs_when_up
count,3.925778e+06,3.925778e+06,3.910973e+06,3.902532e+06,3.902532e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.885487e+06,3.766037e+06,3.883811e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.925778e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.913604e+06,3.925778e+06
mean,5.403311e+05,5.347084e+05,8.868567e+01,-7.793901e-01,5.919715e+00,5.813979e-01,7.294465e-01,8.842978e-01,8.876961e-01,-1.928299e-01,7.733834e-01,1.884104e-02,2.279551e+00,5.051610e+04,9.879194e+04,1.619614e+05,4.998506e+00,5.092058e-01,8.819325e+01,2.211626e+03,6.038816e+00,5.165356e+05,5.142169e+05,5.445410e+05,5.395915e+05,5.568228e+05,5.343705e+05,5.481872e+05,5.326239e+05,3.908339e+01,2.904070e+00,2.291132e+00,2.331207e+00,2.291209e+00,2.331207e+00,2.288409e+00,-1.288765e+02,-4.357640e+00,-2.352538e+00,2.632440e+01,-2.296656e+01,9.812009e-01
std,7.830241e+04,8.158190e+04,6.025909e+00,2.015019e+01,3.368651e+01,4.933300e-01,4.442458e-01,9.672773e-01,8.257246e-01,1.021488e+00,1.013376e+00,2.048049e+01,3.217943e+01,1.585875e+05,2.106543e+05,2.495596e+05,2.666002e+00,4.999153e-01,6.576435e+00,3.349139e+02,5.147727e-01,7.962577e+04,7.851485e+04,7.847754e+04,9.531888e+04,7.987036e+04,9.523073e+04,7.822602e+04,8.938566e+04,2.311444e+01,1.731435e+00,2.589246e+00,2.676471e+00,2.589280e+00,2.676471e+00,5.983404e+00,8.730541e+00,3.076268e+00,1.063525e+01,4.081825e+00,8.880890e+00,8.177113e-01
min,1.125260e+05,1.125260e+05,3.610000e+01,-2.521450e+04,-3.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.013900e+03,-1.600000e+01,-2.642677e+04,-2.848000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,4.130000e+02,-9.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.718084e+01,-7.721720e+02,-7.512015e+01,-1.279061e+02,-8.386989e+01,-1.843371e+02,0.000000e+00
25%,4.771320e+05,4.621010e+05,8.460000e+01,-2.130000e+00,5.630000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-9.000000e-01,3.000000e-01,-5.800000e-01,1.640000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00,8.400000e+01,2.068000e+03,5.700000e+00,4.560780e+05,4.580150e+05,5.148880e+05,4.935960e+05,5.164160e+05,4.600750e+05,4.933160e+05,4.612350e+05,1.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.200430e+00,-1.356030e+02,-6.440000e+00,-1.126000e+01,2.331265e+01,-2.898159e+01,0.000000e+00
50%,5.436060e+05,5.434010e+05,9.000000e+01,-1.480000e+00,5.920000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.000000e-01,9.000000e-01,2.000000e-02,2.260000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e+00,1.000000e+00,8.950000e+01,2.242000e+03,6.000000e+00,5.192220e+05,5.186920e+05,5.439390e+05,5.707990e+05,5.917200e+05,5.424550e+05,5.469910e+05,5.453410e+05,3.900000e+01,3.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,4.009364e+00,-1.307519e+02,-4.531291e+00,-3.020000e+00,2.629849e+01,-2.186600e+01,1.000000e+00
75%,6.051820e+05,5.959090e+05,9.330000e+01,9.400000e-01,6.200000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,5.000000e-01,1.330000e+00,6.100000e-01,2.870000e+00,0.000000e+00,0.000000e+00,4.551170e+05,7.000000e+00,1.000000e+00,9.310000e+01

In [7]:
df.isnull().sum()

game_date                      0
pitch_type                     0
Pitcher_name                   0
pitcher_id                     0
batter_name                    0
batter_id                      0
release_speed              14805
release_pos_x              23246
release_pos_z              23246
stand                          0
p_throws                       0
balls                          0
strikes                        0
pfx_x                      12174
pfx_z                      12174
plate_x                    12174
plate_z                    12174
on_3b                          0
on_2b                          0
on_1b                          0
inning                         0
inning_topbot                  0
effective_speed            40291
release_spin_rate         159741
release_extension          41967
Catcher                        0
FirstBasemen                   0
SecondBasemen                  0
ThirdBasemen                   0
ShortStop                      0
LeftField 

# Baseline

In [8]:
df['strike_attempt'].value_counts(normalize=True)

strike    0.459683
ball      0.359041
out       0.114194
ob        0.067082
Name: strike_attempt, dtype: float64

# Drop Nulls

---
I don't want to deal with it.

In [10]:
df_dropped_release_speed = df[df['release_speed'].notnull()].copy()

In [12]:
df_drop_releasepos_x = df_dropped_release_speed[df_dropped_release_speed['release_pos_x'].notnull()].copy()

In [19]:
df.dropna(subset=['release_speed','release_pos_x','release_extension','effective_speed'], inplace=True)

# Dropped 42287 rows, which is only 1% of data, shouldn't affect 3 million points of data, may impute

# Create `X` and `y`

In [21]:
features = ['Pitcher_name','batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features2 = ['batter_name','pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']
features3 = ['pitch_name','release_speed', 'release_pos_x', 'release_pos_z', 'stand',
       'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z','inning', 'inning_topbot', 'effective_speed',
       'release_spin_rate', 'pitch_number', 'bat_score', 'fld_score', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'outs_when_up']

In [22]:
X = df[features]
y = df['strike_attempt']

## LabelEncode `y`

In [23]:
le = LabelEncoder()

In [24]:
y_encoded = le.fit_transform(y)

In [37]:
y_encoded

array([3, 3, 3, ..., 3, 0, 0])

# Dummify (if needed)

In [34]:
[col for col in X.columns if X[col].dtypes == 'O']

['Pitcher_name', 'batter_name', 'pitch_name']

In [35]:
X_dummy = pd.get_dummies(X, columns=[col for col in X.columns if X[col].dtypes == 'O'])

In [38]:
X_dummy.head()

release_speed  release_pos_x  release_pos_z  stand  p_throws  balls  \
0           96.1          -1.92           5.95      1         1      1   
1           93.1          -1.50           6.02      1         1      1   
2           97.0          -1.37           6.08      1         1      1   
3           93.6          -1.25           6.06      1         1      0   
4           97.1          -1.49           6.08      1         1      0   

   strikes  pfx_x  pfx_z  plate_x  plate_z  inning  inning_topbot  \
0        2  -0.27   1.70    -0.93     1.62      12              0   
1        2   0.24   0.72     0.78     1.55      12              0   
2        2  -0.50   1.41     1.32     2.47      12              0   
3        2   0.32   0.80     2.92     2.23      12              0   
4        1  -0.76   1.63     0.26     2.74      12              0   

   effective_speed  release_spin_rate  pitch_number  bat_score  fld_score  \
0             95.4             2463.0             6          2          7   
1             92.2             2705.0             5          2          7   
2             96.4             2362.0             4          2          7   
3             92.8             2724.0             3          2          7   
4             96.0             2401.0             2          2          7   

      vx0      vy0    vz0     ax      ay      az  outs_when_up  \
0   3.075 -139.609 -9.724 -2.028  34.780 -11.706             2   
1   5.156 -135.442 -7.227  5.004  28.835 -25.323             2   
2   8.459 -140.843 -7.313 -5.412  31.551 -15.189             2   
3  10.094 -135.734 -5.821  6.079  26.355 -24.250             2   
4   6.519 -140.993 -7.138 -9.148  32.251 -11.956             2   

   Pitcher_name_A.J. Achter  Pitcher_name_A.J. Burnett  \
0                         0                          0   
1                         0                          0   
2                         0                          0   
3                         0                          0   
4                         0                          0   

   Pitcher_name_A.J. Cole  Pitcher_name_A.J. Griffin  \
0                       0                          0   
1                       0                          0   
2                       0                          0   
3                       0                          0   
4                       0                          0   

   Pitcher_name_A.J. Minter  Pitcher_name_A.J. Morris  Pitcher_name_A.J. Puk  \
0                         0                         0                      0   
1                         0                         0                      0   
2                         0                         0                      0   
3                         0                         0                      0   
4                         0                         0                      0   

   Pitcher_name_A.J. Schugel  Pitcher_name_AJ Ramos  Pitcher_name_AJ Reed  \
0                          0                      0                     0   
1                          0                      0                     0   
2                          0                      0                     0   
3                          0                      0                     0   
4                          0                      0                     0   

   Pitcher_name_Aaron Altherr  Pitcher_name_Aaron Barrett  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
4                           0                           0   

   Pitcher_name_Aaron Blair  Pitcher_name_Aaron Brooks  \
0                         0                          0   
1                         0                          0   
2                         0                          0   
3                         0                          0

# Modeling

---
Skeleton for setting up model

```python
# Setting up Params
params = {[]}
# Instantitate Randomized, fuck grid
model = RandomizedSearch(REAL MODEL) # Set up to give cross validated score
model.score() # gives cv score
```

In [39]:
# Use X_dummy and y_encoded

## Logistic Regression

In [43]:
# Parameters
params = {
    'penalty':['l1','l2'],
    'solver':['newton-cg','sag','lbfgs','saga'],
    'max_iter':[75, 100, 125],
    'multi_class':['multinomial']
}
# Instantiate
logreg = RandomizedSearchCV(estimator=LogisticRegression(random_state=66),
                            param_distributions=params,
                            verbose=2,
                            n_jobs=7)

In [45]:
logreg.fit(X_dummy, y_encoded)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


MemoryError: Unable to allocate 15.4 GiB for an array with shape (4255, 3883491) and data type uint8